### Installing and Importing Dependencies

In [ ]:
!python -m pip install pennylane pennylane-lightning pennylane-lightning-gpu --upgrade

!pip install custatevec-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
MODEL_SAVE_PATH = "/content/drive/MyDrive/H/QMLExperiments/APTOS_models/"
data_dir = "/content/drive/MyDrive/H/qmlData"
extract_dir = "/content/drive/MyDrive/H/QMLExperiments/G1020"
dataset = "G1020"

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from google.colab import drive
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Sampler
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import time
import os
import copy
from torch.optim.lr_scheduler import CosineAnnealingLR
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, roc_curve,
    precision_recall_curve, auc
)
from sklearn.preprocessing import label_binarize
import random
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pickle
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, f1_score

os.environ["OMP_NUM_THREADS"] = "1"


In [ ]:
import os
os.chdir('/content/drive/MyDrive/H')
from RETFound_MAE import models_vit

Feature Extraction with RETFound

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])


def prepare_model(chkpt_dir, arch='vit_large_patch16', ):
    # build model
    model = models_vit.__dict__[arch](
        img_size=224,
        num_classes=5,
        drop_path_rate=0,
        global_pool=True,
    )
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu', weights_only=False)
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    return model

def run_one_image(img, model):
    x = torch.tensor(img)
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)

    x = x.to(device, non_blocking=True)
    latent = model.forward_features(x.float())
    latent = torch.squeeze(latent)

    return latent

In [ ]:
# download pre-trained RETFound

chkpt_dir = './RETFound_MAE/RETFound_cfp_weights.pth'
model_ = prepare_model(chkpt_dir, 'vit_large_patch16')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_.to(device)
print('Model loaded.')

Model loaded.


In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def feature_extract(data_path, model_):
    model_.eval()

    feature_dicts = {
        'train': {'names': [], 'features': [], 'labels': []},
        'val': {'names': [], 'features': [], 'labels': []},
        'test': {'names': [], 'features': [], 'labels': []}
    }

    for split in ['train', 'val', 'test']:
        split_path = os.path.join(data_path, split)

        for class_name in os.listdir(split_path):
            class_folder = os.path.join(split_path, class_name)

            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)

                img = Image.open(img_path).resize((224, 224))
                img_array = np.array(img) / 255.0
                img_normalized = (img_array - imagenet_mean) / imagenet_std

                x = torch.tensor(img_normalized)
                x = x.unsqueeze(dim=0)
                x = torch.einsum('nhwc->nchw', x)
                x = x.to(device, non_blocking=True)

                with torch.no_grad():
                    latent = model_.forward_features(x.float())
                    latent = torch.squeeze(latent)

                feature_dicts[split]['names'].append(img_name)
                feature_dicts[split]['features'].append(latent.detach().cpu().numpy())
                feature_dicts[split]['labels'].append(class_name)

    pickle.dump(feature_dicts['train'], open('G1020_train1.pkl', 'wb'))
    pickle.dump(feature_dicts['val'], open('G1020_val1.pkl', 'wb'))
    pickle.dump(feature_dicts['test'], open('G1020_test1.pkl', 'wb'))

    print("Feature extraction completed.")
    return feature_dicts

def load_features(pkl_path='Feature_latent_multiclass.pkl'):
    with open(pkl_path, 'rb') as f:
        features = pickle.load(f)
    return features

In [ ]:
feature_extract(extract_dir, model_)

Feature extraction completed.


{'train': {'names': ['image_1139.jpg',
   'image_1144.jpg',
   'image_1136.jpg',
   'image_1093.jpg',
   'image_1088.jpg',
   'image_1092.jpg',
   'image_1138.jpg',
   'image_10.jpg',
   'image_1094.jpg',
   'image_1142.jpg',
   'image_1141.jpg',
   'image_1140.jpg',
   'image_1095.jpg',
   'image_1087.jpg',
   'image_1217.jpg',
   'image_1332.jpg',
   'image_1254.jpg',
   'image_1285.jpg',
   'image_1199.jpg',
   'image_1322.jpg',
   'image_1255.jpg',
   'image_1160.jpg',
   'image_1165.jpg',
   'image_1189.jpg',
   'image_1216.jpg',
   'image_1330.jpg',
   'image_1204.jpg',
   'image_1250.jpg',
   'image_1187.jpg',
   'image_1213.jpg',
   'image_1333.jpg',
   'image_1197.jpg',
   'image_1214.jpg',
   'image_1203.jpg',
   'image_1161.jpg',
   'image_1164.jpg',
   'image_1215.jpg',
   'image_1260.jpg',
   'image_1188.jpg',
   'image_1218.jpg',
   'image_1190.jpg',
   'image_143.jpg',
   'image_1414.jpg',
   'image_1413.jpg',
   'image_144.jpg',
   'image_1426.jpg',
   'image_1507.jpg',

In [ ]:
num_qubits = 4
batch_size = 4
device = qml.device("default.qubit", wires=num_qubits)

In [ ]:
def load_and_shuffle_data(seed=42, num_qubits=4):
    with open(dataset+'_train1.pkl', 'rb') as f:
        train_data = pickle.load(f)
    with open(dataset+'_val1.pkl', 'rb') as f:
        val_data = pickle.load(f)
    with open(dataset+'_test1.pkl', 'rb') as f:
        test_data = pickle.load(f)

    train_features = np.array(train_data['features'])
    val_features = np.array(val_data['features'])
    test_features = np.array(test_data['features'])

    train_labels = np.array(train_data['labels'])
    val_labels = np.array(val_data['labels'])
    test_labels = np.array(test_data['labels'])

    train_features, train_labels = shuffle(train_features, train_labels, random_state=seed)
    val_features, val_labels = shuffle(val_features, val_labels, random_state=seed)
    test_features, test_labels = shuffle(test_features, test_labels, random_state=seed)

    label_encoder = LabelEncoder()
    label_encoder.fit(np.concatenate([train_labels, val_labels, test_labels]))

    train_labels_encoded = label_encoder.transform(train_labels)
    val_labels_encoded = label_encoder.transform(val_labels)
    test_labels_encoded = label_encoder.transform(test_labels)

    pca = PCA(n_components=num_qubits)
    pca.fit(train_features)

    train_features_reduced = pca.transform(train_features)
    val_features_reduced = pca.transform(val_features)
    test_features_reduced = pca.transform(test_features)

    scaler = StandardScaler()
    scaler.fit(train_features_reduced)

    train_features_scaled = scaler.transform(train_features_reduced)
    val_features_scaled = scaler.transform(val_features_reduced)
    test_features_scaled = scaler.transform(test_features_reduced)

    return (train_features_scaled, train_labels_encoded,
            val_features_scaled, val_labels_encoded,
            test_features_scaled, test_labels_encoded,
            label_encoder.classes_)



In [ ]:
def run_experiment(model_func, dataset_name, num_qubits=4, seed=42, output_dir=data_dir):
    model_name = model_func.__name__.replace("run_", "")
    is_quantum = model_name.startswith("q")
    print(f"Running {model_name.upper()} experiment on {dataset_name} dataset")

    print(f"Loading and preprocessing data with seed {seed}...")
    X_train, y_train, X_val, y_val, X_test, y_test, class_names = load_and_shuffle_data(
        seed=seed, num_qubits=num_qubits)

    print(f"Training and evaluating {model_name.upper()}...")
    if is_quantum:
        model_result = model_func(X_train, y_train, X_val, y_val, X_test, y_test, num_qubits=num_qubits)
    else:
        model_result = model_func(X_train, y_train, X_val, y_val, X_test, y_test)

    result = {
        'model': model_name,
        'dataset': dataset_name,
        'seed': seed,
        'class_names': class_names,
        'y_test': y_test,
        'is_quantum': is_quantum,
        'num_qubits': num_qubits if is_quantum else None,
        **model_result
    }

    print(f"\n{model_name.upper()} Results:")
    print("  Validation Metrics:")
    print(f"    Accuracy:  {model_result['val_accuracy']:.4f}")
    print(f"    Precision: {model_result['val_precision']:.4f}")
    print(f"    Recall:    {model_result['val_recall']:.4f}")
    print(f"    F1 Score:  {model_result['val_f1']:.4f}")
    print("  Test Metrics:")
    print(f"    Accuracy:  {model_result['accuracy']:.4f}")
    print(f"    Precision: {model_result['precision']:.4f}")
    print(f"    Recall:    {model_result['recall']:.4f}")
    print(f"    F1 Score:  {model_result['f1']:.4f}")

    output_file = os.path.join(output_dir, f'{model_name}_{dataset_name}_results.pkl')
    with open(output_file, 'wb') as f:
        pickle.dump(result, f)
    print(f"Results saved to {output_file}")

    return result


In [ ]:
def run_qsvm(X_train, y_train, X_val, y_val, X_test, y_test, num_qubits=4):
    dev = qml.device("lightning.gpu", wires=num_qubits)

    @qml.qnode(dev)
    def qksvm_kernel_cirq(a, b):
        qml.AngleEmbedding(a, wires=range(num_qubits))
        qml.adjoint(qml.AngleEmbedding(b, wires=range(num_qubits)))
        return qml.probs(wires=range(num_qubits))

    def quantum_kernel_pca(A, B):
        return np.array([[qksvm_kernel_cirq(a, b)[0] for b in B] for a in A])

    svm = SVC(kernel='precomputed', class_weight='balanced', probability=True)

    kernel_train = quantum_kernel_pca(X_train, X_train)
    kernel_val = quantum_kernel_pca(X_train, X_val)
    kernel_test = quantum_kernel_pca(X_train, X_test)

    svm.fit(kernel_train, y_train)

    val_pred = svm.predict(kernel_val.T)
    val_accuracy = accuracy_score(y_val, val_pred)
    val_precision = precision_score(y_val, val_pred, average='weighted')
    val_recall = recall_score(y_val, val_pred, average='weighted')
    val_f1 = f1_score(y_val, val_pred, average='weighted')

    y_pred = svm.predict(kernel_test.T)
    y_prob = svm.predict_proba(kernel_test.T)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'y_pred': y_pred,
        'y_prob': y_prob,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    }

In [ ]:
def run_svm(X_train, y_train, X_val, y_val, X_test, y_test):
    print("Training SVM...")
    svm = SVC(kernel='rbf', class_weight='balanced', probability=True)
    svm.fit(X_train, y_train)

    print("Computing validation predictions...")
    val_pred = svm.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_pred)
    val_precision = precision_score(y_val, val_pred, average='weighted')
    val_recall = recall_score(y_val, val_pred, average='weighted')
    val_f1 = f1_score(y_val, val_pred, average='weighted')

    print("Computing test predictions...")
    y_pred = svm.predict(X_test)

    y_prob = svm.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'y_pred': y_pred,
        'y_prob': y_prob,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    }


In [ ]:
def quantum_feature_map(x, num_qubits=4):
    device = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(device)
    def feature_map_circuit():
        for i in range(min(len(x), num_qubits)):
            qml.RX(x[i], wires=i)
        return [qml.expval(qml.PauliZ(i)) for i in range(min(len(x), num_qubits))]

    return feature_map_circuit()

def qknn_fit(X_train, y_train, k=5, num_qubits=4):
    quantum_X_train = np.array([quantum_feature_map(x, num_qubits) for x in X_train])

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(quantum_X_train, y_train)

    return knn

def qknn_predict(knn, X_test, num_qubits=4):
    quantum_X_test = np.array([quantum_feature_map(x, num_qubits) for x in X_test])

    return knn.predict(quantum_X_test)

def qknn_predict_proba(knn, X_test, num_qubits=4):
    quantum_X_test = np.array([quantum_feature_map(x, num_qubits) for x in X_test])

    return knn.predict_proba(quantum_X_test)

def run_qknn(X_train, y_train, X_val, y_val, X_test, y_test, num_qubits=4):
    print("Training Quantum KNN...")
    knn = qknn_fit(X_train, y_train, k=5, num_qubits=num_qubits)

    print("Computing validation predictions...")
    val_pred = qknn_predict(knn, X_val, num_qubits)
    val_accuracy = accuracy_score(y_val, val_pred)
    val_precision = precision_score(y_val, val_pred, average='weighted')
    val_recall = recall_score(y_val, val_pred, average='weighted')
    val_f1 = f1_score(y_val, val_pred, average='weighted')

    print("Computing test predictions...")
    y_pred = qknn_predict(knn, X_test, num_qubits)

    y_prob = qknn_predict_proba(knn, X_test, num_qubits)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'y_pred': y_pred,
        'y_prob': y_prob,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    }

In [ ]:
def run_knn(X_train, y_train, X_val, y_val, X_test, y_test):
    knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
    knn.fit(X_train, y_train)

    val_pred = knn.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_pred)
    val_precision = precision_score(y_val, val_pred, average='weighted')
    val_recall = recall_score(y_val, val_pred, average='weighted')
    val_f1 = f1_score(y_val, val_pred, average='weighted')

    y_pred = knn.predict(X_test)
    y_prob = knn.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'y_pred': y_pred,
        'y_prob': y_prob,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    }


In [ ]:
qsvm_result = run_experiment(run_qsvm, dataset, num_qubits=4)

Running QSVM experiment on G1020 dataset
Loading and preprocessing data with seed 42...
Training and evaluating QSVM...

QSVM Results:
  Validation Metrics:
    Accuracy:  0.4909
    Precision: 0.6445
    Recall:    0.4909
    F1 Score:  0.5280
  Test Metrics:
    Accuracy:  0.4869
    Precision: 0.5687
    Recall:    0.4869
    F1 Score:  0.5064
Results saved to /content/drive/MyDrive/H/qmlData/qsvm_G1020_results.pkl


In [ ]:
svm_result = run_experiment(run_svm, dataset)

Running SVM experiment on G1020 dataset
Loading and preprocessing data with seed 42...
Training and evaluating SVM...
Training SVM...
Computing validation predictions...
Computing test predictions...

SVM Results:
  Validation Metrics:
    Accuracy:  0.4848
    Precision: 0.6554
    Recall:    0.4848
    F1 Score:  0.5205
  Test Metrics:
    Accuracy:  0.4967
    Precision: 0.5749
    Recall:    0.4967
    F1 Score:  0.5160
Results saved to /content/drive/MyDrive/H/qmlData/svm_G1020_results.pkl


In [ ]:
knn_result = run_experiment(run_knn, dataset)

Running KNN experiment on G1020 dataset
Loading and preprocessing data with seed 42...
Training and evaluating KNN...

KNN Results:
  Validation Metrics:
    Accuracy:  0.6909
    Precision: 0.6266
    Recall:    0.6909
    F1 Score:  0.6520
  Test Metrics:
    Accuracy:  0.6046
    Precision: 0.5380
    Recall:    0.6046
    F1 Score:  0.5613
Results saved to /content/drive/MyDrive/H/qmlData/knn_G1020_results.pkl


In [ ]:
qknn_result = run_experiment(run_qknn, dataset, num_qubits=4)

Running QKNN experiment on G1020 dataset
Loading and preprocessing data with seed 42...
Training and evaluating QKNN...
Training Quantum KNN...
Computing validation predictions...
Computing test predictions...

QKNN Results:
  Validation Metrics:
    Accuracy:  0.6727
    Precision: 0.6255
    Recall:    0.6727
    F1 Score:  0.6455
  Test Metrics:
    Accuracy:  0.6013
    Precision: 0.5400
    Recall:    0.6013
    F1 Score:  0.5619
Results saved to /content/drive/MyDrive/H/qmlData/qknn_G1020_results.pkl
